In [1]:
import re

rdd = sc.textFile("./access_log")

In [2]:
#Parse file and filter not 2XX status code records

def parseLogs(record):
    ip = record[:record.find(' ')]
    
    date = record[record.find('[') + 1:
                  record.find(':')]
    
    time = record[record.find(':') + 1:
                  record.find(' ', record.find(':'))]
    
    begginingOfURL = record.find('/', record.find(']'))
    
    url = record[begginingOfURL:
                 record.rfind(' ', begginingOfURL, record.rfind('"'))]
    
    statusCode = record[record.rfind(' ', 0, record.rfind(' ')) + 1:
                        record.rfind(' ')]
    
    return (url, [ip, date, time, statusCode])

def filterOK(record):
    return int(record[1][3]) // 100 == 2 

filteredRDD = rdd.map(parseLogs).filter(filterOK)

In [3]:
#Count URL accesses
countByURLRDD = filteredRDD.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)
moreThan10URLRDD = countByURLRDD.filter(lambda x: x[1] > 10)

moreThan10URLRDD.sortByKey().repartition(1).saveAsTextFile("LogCountsByURL")

In [4]:
#Count by month accesses for year 2010
countByMonthRDD = filteredRDD.filter(lambda x: int(x[1][1].split('/')[2]) == 2010) \
                                .map(lambda x: (x[1][1].split('/')[1], 1)) \
                                .reduceByKey(lambda a, b: a + b)

countByMonthRDD.sortByKey().repartition(1).saveAsTextFile("LogCountsByMonth")

In [7]:
#Count by hour accesses for year 2010
countByHourRDD = filteredRDD.filter(lambda x: int(x[1][1].split('/')[2]) == 2010) \
                                .map(lambda x: (x[1][2].split(':')[0], 1)) \
                                .reduceByKey(lambda a, b: a + b)

countByHourRDD.sortByKey().repartition(1).saveAsTextFile("LogCountsByHour")